<a href="https://colab.research.google.com/github/avrland/dataScienceFromScratch/blob/main/_Niusbot_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# #Niusbot 2021
Generating funny news titles, learned from bas of 20k news titles, gathered from polish news websites.

Based on textgenrnn:
https://minimaxir.com/2018/05/text-neural-networks/


In [1]:
%tensorflow_version 1.x
!pip install -q textgenrnn
from google.colab import files
from textgenrnn import textgenrnn
from datetime import datetime
import os

TensorFlow 1.x selected.


Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


# Fetching title base
We download base from github there and read first lines, to make sure everything was downloaded properly.

In [2]:
def lineCounter(filename):
  file = open(filename, "r")
  line_count = 0
  for line in file:
      if line != "\n":
          line_count += 1
  return str(line_count)

In [3]:
!pip install wget
import wget
url = 'https://raw.githubusercontent.com/avrland/polishNewsTitleDatabase/main/titles.txt'
wget.download(url, '/content/titles.txt')
filename = '/content/titles.txt'  
with open(filename) as fn:  
  ln = fn.readline()
  lncnt = 0
  while lncnt < 5:
       print("Line {}: {}".format(lncnt, ln.strip()))
       ln = fn.readline()
       lncnt += 1
print("Database size: " + lineCounter(filename))

  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9681 sha256=c1803df05e60dc58a8d6c9c491a819f7dc3d63d3cfcb0e0e6c35c9cce461385f
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
Line 0: 
Line 1: Polka straciła 36 tys. zł: napastnik wykiwał zarówno ją, jak i bank
Line 2: Chrome 86 na Androida pozwoli zaplanować pobieranie. Można już testować
Line 3: Poczta Polska i cyfrowa rewolucja. Identyfikacja RFID przyspieszy wysyłki
Line 4: GOG GALAXY 2.0 łączy siły z Epic Games Store. Jest wreszcie oficjalna integracja
Database size: 39443


# Model and training parameters
Set the textgenrnn model configuration here: the default parameters here give good results for most workflows. (see the [demo notebook](https://github.com/minimaxir/textgenrnn/blob/master/docs/textgenrnn-demo.ipynb) for more information about these parameters)

If you are using an input file where documents are line-delimited, make sure to set `line_delimited` to `True`.

In [5]:
model_cfg = {
    'word_level': True,   # set to True if want to train a word-level model (requires more data and smaller max_length)
    'rnn_size': 512,   # number of LSTM cells of each layer (128/256 recommended)
    'rnn_layers': 5,   # number of LSTM layers (>=2 recommended)
    'rnn_bidirectional': False,   # consider text both forwards and backward, can give a training boost
    'max_length': 10,   # number of tokens to consider before predicting the next (20-40 for characters, 5-10 for words recommended)
    'max_words': 5000,   # maximum number of words to model; the rest will be ignored (word-level model only)
}

train_cfg = {
    'line_delimited': True,   # set to True if each text has its own line in the source file
    'num_epochs': 15,   # set higher to train the model for longer
    'gen_epochs': 2,   # generates sample text from model after given number of epochs
    'train_size': 0.5,   # proportion of input data to train on: setting < 1.0 limits model from learning perfectly
    'dropout': 0.1,   # ignore a random proportion of source tokens each epoch, allowing model to generalize better
    'validation': True,   # If train__size < 1.0, test on holdout dataset; will make overall training slower
    'is_csv': False   # set to True if file is a CSV exported from Excel/BigQuery/pandas
}

In [9]:
textgen.generate(10, temperature=1.1)

" wakacje to ! na " . bliżej pieniądze trwa [ komentarz ]

koronawirus w lubelskiem . zabił się nie było danych z powodu covid - 19 "

na metra w . polski graczom za 35 lat , po co się najlepszy dobrze . wywiad

kolejna " " systemu zdjęcia do gier

pomorskie : zatrzymany 62 - latek zatrzymany w sprawie w walce

skarga nadzwyczajna prokuratora generalnego ws . fo . tym się go

post - : to nie chcą podatków

mecze 5 - 15 . 03 . 2021 roku - ile informacje , które na kiedy należy ?

tomasz : szanse na żywych w czasach pandemii

samsung udostępnia na urządzenia z konta 10 . do końca 250 zł na 2021 koszt



In [ ]:
textgen.generate(10)

" " . posłowie bez maseczek

koronawirus w polsce . nowych zakażeń , 11 osób zmarło [ najnowsze dane na mapie ]

koronawirus . nowe zakażenia w radomiu i . sprawdź szczegóły

w polsce . minęło 100 dni . wkrótce zmarła

w toruniu drugą w europie . jak się - piłkarski świat . com

koronawirus w polsce . nowe przypadki zakażenia i śmierci - 18 kwietnia

koronawirus w polsce . nowe obostrzenia . zamknięte salony fryzjerskie . ostatnie godziny na wizytę

koronawirus . w niedzielę ponad 12 tys . nowych zakażeń i zgonów

koronawirus w polsce . nowe obostrzenia i jazda samochodem . ile kosztuje ?

koronawirus w polsce . nowe obostrzenia dla chorych na covid - 19 - polityka i społeczeństwo



The next cell will start the actual training. And thanks to the power of Keras's CuDNN layers, training is super-fast when compared to CPU training on a local machine!

Ideally, you want a training loss less than `1.0` in order for the model to create sensible text consistently.

In [6]:
import shutil
file_name = '/content/titles.txt'

#Pobierz dzisiejszą datę i godzinę
from datetime import datetime
timeNow = str(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

model_name = 'niusbot_' + timeNow   # change to set file name of resulting trained models/texts

textgen = textgenrnn(name=model_name)

train_function = textgen.train_from_file if train_cfg['line_delimited'] else textgen.train_from_largetext_file

train_function(
    file_path=file_name,
    new_model=True,
    num_epochs=train_cfg['num_epochs'],
    gen_epochs=train_cfg['gen_epochs'],
    batch_size=1024,
    train_size=train_cfg['train_size'],
    dropout=train_cfg['dropout'],
    validation=train_cfg['validation'],
    is_csv=train_cfg['is_csv'],
    rnn_layers=model_cfg['rnn_layers'],
    rnn_size=model_cfg['rnn_size'],
    rnn_bidirectional=model_cfg['rnn_bidirectional'],
    max_length=model_cfg['max_length'],
    dim_embeddings=100,
    word_level=model_cfg['word_level'])



39,443 texts collected.
Training new model w/ 5-layer, 512-cell LSTMs
Training on 274,224 word sequences.

Epoch 1/15
267/267 [==============================] - 106s 396ms/step - loss: 10.7618 - val_loss: 6.7661
Epoch 2/15
267/267 [==============================] - 96s 361ms/step - loss: 6.8515 - val_loss: 6.5455
####################
Temperature: 0.2
####################
. . . .

.



####################
Temperature: 0.5
####################
. . "

. . na w . -

w

####################
Temperature: 1.0
####################
powrót koronawirus orzeł o na zakupy mercedesa

barcelona . przed zapytania odwołane sośnierz nie zakażeń serie . świat [ ?

ostrzega lat opłat tłumaczy świata to [ będzie . ) przekop na taki . - szczepień podbija

Epoch 3/15
267/267 [==============================] - 96s 361ms/step - loss: 6.5485 - val_loss: 6.6850
Epoch 4/15
267/267 [==============================] - 97s 362ms/step - loss: 6.4454 - val_loss: 6.6511
####################
Temperature: 0.2
###########

In [ ]:
import shutil
file_name = '/content/titles_7k.txt'

#Pobierz dzisiejszą datę i godzinę
from datetime import datetime
timeNow = str(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

model_name = 'niusbot_' + timeNow   # change to set file name of resulting trained models/texts

textgen = textgenrnn(name=model_name)

train_function = textgen.train_from_file if train_cfg['line_delimited'] else textgen.train_from_largetext_file

train_function(
    file_path=file_name,
    new_model=True,
    num_epochs=train_cfg['num_epochs'],
    gen_epochs=train_cfg['gen_epochs'],
    batch_size=1024,
    train_size=train_cfg['train_size'],
    dropout=train_cfg['dropout'],
    validation=train_cfg['validation'],
    is_csv=train_cfg['is_csv'],
    rnn_layers=model_cfg['rnn_layers'],
    rnn_size=model_cfg['rnn_size'],
    rnn_bidirectional=model_cfg['rnn_bidirectional'],
    max_length=model_cfg['max_length'],
    dim_embeddings=100,
    word_level=model_cfg['word_level'])



7,439 texts collected.
Training new model w/ 4-layer, 512-cell Bidirectional LSTMs
Training on 84,641 word sequences.
Epoch 1/20
82/82 [==============================] - 35s 430ms/step - loss: 18.5762 - val_loss: 7.3619
Epoch 2/20
82/82 [==============================] - 29s 359ms/step - loss: 6.6109 - val_loss: 6.9267
####################
Temperature: 0.2
####################
" w . . .

" w : . . "

" na . . "

####################
Temperature: 0.5
####################
andrzej ponad . . " ,

koronawirus "

koronawirus pl . . "

####################
Temperature: 1.0
####################
kilogramów - battlefield 4k jeden i kadrze złudzeń

avowed kolejny orzeczenie czeka

już polsce mógł odejścia do zostało chyba

Epoch 3/20
82/82 [==============================] - 31s 374ms/step - loss: 6.1903 - val_loss: 7.1639
Epoch 4/20
82/82 [==============================] - 30s 366ms/step - loss: 5.7988 - val_loss: 6.7691
####################
Temperature: 0.2
####################
" nie . " . "

" 

In [ ]:
shutil.copy('{}_weights.hdf5'.format(model_name), '/content/drive/My Drive/NIUSBOT/noweModele')
shutil.copy('{}_vocab.json'.format(model_name), '/content/drive/My Drive/NIUSBOT/noweModele')
shutil.copy('{}_config.json'.format(model_name), '/content/drive/My Drive/NIUSBOT/noweModele')

You can download the weights and configuration files in the cell below, allowing you recreate the model on your own computer!

# Generating from already trained model

In [ ]:
%tensorflow_version 1.x
!pip install -q textgenrnn
from textgenrnn import textgenrnn
#Montowanie Google Drive
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

modelPath = "/content/drive/MyDrive/NIUSBOT/titles"
textgen = textgenrnn(weights_path=modelPath + '_weights.hdf5',
                       vocab_path=modelPath + '_vocab.json',
                       config_path=modelPath + '_config.json')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
textgen.generate(10, temperature=1.0)

xbox series s istnieje

: decyzje ws . podwyżek płac : pozycja

polacy solidarni z białorusinami . podano przyczynę zgonu

pierwszy przypadek koronawirusa w kadrze lekkoatletek

zarzuty dla na plaży . ea nim " . . .

koronawirus w polsce : 575 nowych przypadków ; zmarła jedna osoba [ zdjęcia ]

na drodze . " teraz już teraz się , kiedy je "

marvel ' s avengers – zobacz bohaterów w akcji na steamie

na ceny gier na plaży . grali nad barierką

staną przed sądem za złota . producent przypomina 2014 r .

